### Prediction from Validation dataset

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os

# Load the saved model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1000)  # Adjust to match the original model's output units
model.load_state_dict(torch.load('flower_classification_model.pth'))
model.eval()

# Create a new model with the correct final layer
new_model = models.resnet18(pretrained=True)
new_model.fc = nn.Linear(new_model.fc.in_features, 2)  # Adjust to match the desired output units

# Copy the weights and biases from the loaded model to the new model
new_model.fc.weight.data = model.fc.weight.data[0:2]  # Copy only the first 2 output units
new_model.fc.bias.data = model.fc.bias.data[0:2]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import datetime

start_time = datetime.datetime.now()
# Display the image with the predicted class name
def image_show(image):
    image = np.array(image)
    plt.imshow(image)
    plt.axis('off')
    plt.text(10, 10, f'Predicted: {predicted_class_name}', fontsize=12, color='white', backgroundcolor='red')
    plt.show()

total_correct = 0
total_images = 0

# Iterate through the dataset
for true_class in os.listdir('dataset/val/'):  # Assuming dataset is a list of tuples containing image paths and true labels
    for file_name in os.listdir(f'dataset/val/{true_class}'):
        image = Image.open(f'dataset/val/{true_class}/{file_name}')

        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0)
    
        # Perform inference
        with torch.no_grad():
            output = model(input_batch)
        
        # Get the predicted class
        _, predicted_class = output.max(1)
        
        # Check if prediction is correct
        predicted_class_name = class_names[predicted_class.item()]
        if predicted_class_name == true_class:
            image_show(image)
            total_correct += 1
        
        total_images += 1

end_time = datetime.datetime.now()